In [1]:
import caffe
import numpy as np
import lmdb

In [2]:
a = np.load("X51.npy")
print a.shape, a.dtype, a.nbytes

(10000, 10000) float64 800000000


In [3]:
a = a.astype(np.float32)
print a.nbytes

400000000


In [4]:
X = a.reshape(10000,1,100,100)
print X.shape

(10000, 1, 100, 100)


In [5]:
y = np.load("y51.npy")
y = y.astype(np.int32)
print y.shape,y,y.nbytes

(10000, 10000) [[21  3 10 ..., 11  9 21]
 [21 10 13 ..., 13  5 21]
 [21  9 11 ..., 10  6 21]
 ..., 
 [21  1  9 ..., 12  9 21]
 [21  9 10 ...,  8  4 21]
 [21  5  8 ..., 10  3 21]] 400000000


In [6]:
y = y.reshape(10000,1,100,100)

In [7]:
print y.shape

(10000, 1, 100, 100)


In [8]:
print X.nbytes

400000000


In [9]:
map_size = 10*a.nbytes
env = lmdb.open('mylmdb',map_size=map_size)

In [10]:
with env.begin(write=True) as txn:
    for i in range(a.shape[0]):
        datum = caffe.proto.caffe_pb2.Datum()
        datum.channels = X.shape[1]
        datum.height = X.shape[2]
        datum.width = X.shape[3]
        datum.data = X[i].tobytes()
        datum.label = 0
        str_id = '{:08}'.format(i)

In [11]:
env = lmdb.open('mylmdb', readonly=True)
with env.begin() as txn:
    raw_datum = txn.get(b'00000000')

datum = caffe.proto.caffe_pb2.Datum()
datum.ParseFromString(raw_datum)

flat_x = np.fromstring(datum.data, dtype=np.uint8)
x = flat_x.reshape(datum.channels, datum.height, datum.width)
y = datum.label

TypeError: expected a readable buffer object

In [8]:
import caffe
import numpy as np
import lmdb
x = np.load("X51.npy")
y = np.load("y51.npy")
x = x.astype(np.float32).reshape(10000,1,100,100)
y = y.astype(np.float32).reshape(10000,1,100,100)
in_db = lmdb.open('data-lmdb', map_size=int(x.nbytes*10))
with in_db.begin(write=True) as in_txn:
    for i in range(x.shape[0]):
        im = x[i]
        im_dat = caffe.io.array_to_datum(im)
        in_txn.put('{:0>10d}'.format(i), im_dat.SerializeToString())
in_db.close()

In [9]:
r_db = lmdb.open('data-lmdb', readonly = True)
r_txn = r_db.begin()
r_cursor = r_txn.cursor()
datum = caffe.proto.caffe_pb2.Datum()
for key, value in r_cursor:
    i = 1
    datum.ParseFromString(value)
    data = caffe.io.datum_to_array(datum)
    print data.shape
    print datum.channels
    print data
    if i == 1:
        break

(1, 100, 100)
1
[[[  0.           1.67448342   7.14212561 ...,  42.58302689  44.62506866
    45.75233841]
  [  3.39622021   0.           5.46764231 ...,  45.97924805  48.02128983
    49.14855957]
  [  4.83443928   1.43821895   0.         ...,  42.39348984  44.43553162
    45.56280518]
  ..., 
  [ 39.90295029  40.71175003  46.17938995 ...,   0.           2.04204273
     3.16931367]
  [ 40.93133545  41.74013138  47.20777512 ...,   2.20748711   0.
     1.12727094]
  [ 39.80406189  40.61286163  46.08050537 ...,   8.57852364   6.37103653
     0.        ]]]


In [10]:
print x[0]

[[[  0.           1.67448342   7.14212561 ...,  42.58302689  44.62506866
    45.75233841]
  [  3.39622021   0.           5.46764231 ...,  45.97924805  48.02128983
    49.14855957]
  [  4.83443928   1.43821895   0.         ...,  42.39348984  44.43553162
    45.56280518]
  ..., 
  [ 39.90295029  40.71175003  46.17938995 ...,   0.           2.04204273
     3.16931367]
  [ 40.93133545  41.74013138  47.20777512 ...,   2.20748711   0.
     1.12727094]
  [ 39.80406189  40.61286163  46.08050537 ...,   8.57852364   6.37103653
     0.        ]]]
